In [1]:
from functools import reduce

from cycler import cycler
from dateutil.relativedelta import relativedelta
import matplotlib.pyplot as plt
from numbers import Number
import numpy as np
import pandas as pd
#from scipy import stats as sstats
import wmfdata as wmf
from wmfdata.utils import pct_str, sig_figs
from wmfdata.charting import pct_fmt, M_fmt, comma_fmt

import gspread
from google.oauth2.service_account import Credentials

You are using wmfdata v1.2, but v1.3.1 is available.

To update, run `pip install --upgrade git+https://github.com/wikimedia/wmfdata-python.git@release`.

To see the changes, refer to https://github.com/wikimedia/wmfdata-python/blob/release/CHANGELOG.md


In [2]:
FILE = "metrics/metrics.tsv"
metrics = pd.read_csv(FILE, sep="\t", parse_dates=["month"]).set_index("month")

In [3]:
metrics

,structured_data_used,num_commons_files_used_content_pages,wikidata_items,wikidata_items_being_reused
month,,,,
2020-07-01,102540679.0,33182261.0,NaN,NaN
2020-08-01,104565582.0,33576054.0,88776242.0,18390627.0
2020-09-01,105631168.0,33815277.0,89386201.0,18472082.0
2020-10-01,106729592.0,32666244.0,90190671.0,18549227.0
2020-11-01,108116779.0,31481821.0,90821857.0,18673974.0
2020-12-01,109284793.0,30628535.0,91480812.0,18782123.0
2021-01-01,110336427.0,30922629.0,91898784.0,18890176.0
2021-02-01,111537756.0,31236223.0,92346206.0,18961132.0
2021-03-01,113749241.0,31994067.0,92766054.0,19081836.0


# Report

In [4]:
report_order = [
    'structured_data_used', 
    'num_commons_files_used_content_pages',
    'wikidata_items',
    'wikidata_items_being_reused'
]

In [5]:
new_metrics = metrics.iloc[[-1]].transpose().reindex(report_order).fillna("—")

In [6]:
new_metrics.at['structured_data_used','status'] =  new_metrics.iloc[0][0]/87000000-1
new_metrics.at['num_commons_files_used_content_pages','status'] = new_metrics.iloc[1][0]/31200000-1
new_metrics.at['wikidata_items_being_reused','status'] = new_metrics.iloc[3][0]/new_metrics.iloc[2][0]

In [7]:
new_metrics

month,2021-11-01 00:00:00,status
structured_data_used,123918750.0,0.424353
num_commons_files_used_content_pages,36208503.0,0.160529
wikidata_items,95420570.0,NaN
wikidata_items_being_reused,20031718.0,0.209931


# Update Google Sheet

In [8]:
FILE = "metrics/metrics.tsv"
all_metrics = pd.read_csv(FILE, sep="\t")

In [9]:
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']

credentials = Credentials.from_service_account_file('big-mender-274521-324384a764e2.json', scopes=scope)
gc = gspread.authorize(credentials)

In [10]:
all_metrics.iloc[-1,1:] = all_metrics.iloc[-1,1:].astype(float)

In [11]:
@property
def sheet1(self):
    """Shortcut property for getting the first worksheet."""
    return self.get_worksheet(0)

In [12]:
wks1 = gc.open("platform evolution/structured data metrics").sheet1
new_metrics = all_metrics.iloc[-1,:].tolist()
wks1.append_row(new_metrics)

{'spreadsheetId': '12pI5UtJ9QBsFWxVjmRVJj6bBdwk_bwxFPmS-APLSQL4',
 'tableRange': 'Sheet1!A1:E17',
 'updates': {'spreadsheetId': '12pI5UtJ9QBsFWxVjmRVJj6bBdwk_bwxFPmS-APLSQL4',
  'updatedRange': 'Sheet1!A18:E18',
  'updatedRows': 1,
  'updatedColumns': 5,
  'updatedCells': 5}}